<a href="https://colab.research.google.com/github/linqus/nlp-huggingface/blob/main/notebooks/unit7/nlp_7_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install datasets transformers[sentencepiece] sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.8 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

raw_datasets = load_dataset('kde4', lang1='en', lang2='fr')

raw_datasets

Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [11]:
split_datasets = raw_datasets['train'].train_test_split(test_size=0.1, shuffle=True, seed=20)
split_datasets['validate'] = split_datasets.pop('test')
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validate: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [12]:
split_datasets['train'][1]

{'id': '152754',
 'translation': {'en': 'Default to expanded threads',
  'fr': 'Par défaut, développer les fils de discussion'}}

In [15]:
from transformers import pipeline

model_checkpoint = 'Helsinki-NLP/opus-mt-en-fr'

translator = pipeline('translation', model_checkpoint)
outputs = translator(split_datasets['train'][1]['translation']['en'])
outputs

[{'translation_text': 'Par défaut pour les threads élargis'}]

In [43]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors='pt')

en_inputs = tokenizer(split_datasets['train'][1]['translation']['en'])
en_inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [44]:
wrong_inputs = tokenizer(split_datasets['train'][1]['translation']['fr'])


In [45]:
full_inputs = tokenizer(split_datasets['train'][1]['translation']['en'], text_target=split_datasets['train'][1]['translation']['fr'])
full_inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [46]:
print('wrong fr: ', tokenizer.convert_ids_to_tokens(wrong_inputs['input_ids']))
print('good fr: ', tokenizer.convert_ids_to_tokens(full_inputs['labels']))

wrong fr:  ['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
good fr:  ['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']
